# Poker-Style Voting through Monday.com API, Google Cloud API, and Github Image Linking
## A Project for Operations at LKCM Headwater

In [ ]:
### Author: Jack Sharp
### Contact: sharpj.@uchicago.edu , jack.a.sharp2@gmail.com

### First, We need to install all of the used packages. Intention is to run this as simply a .py file on anyone's computer, so we will need to pip install all of these (at least the super-non-standard ones) so that we can be sure not to reach an error on import.

In [ ]:
import subprocess
import sys
import ensurepip

try:
    import pip
except ImportError:
    ensurepip.bootstrap()

# Upgrade pip (optional but recommended)
subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", "pip"])



packages = [
    "requests", "pandas", "gspread", "oauth2client",
    "matplotlib", "plotly", "seaborn"
]

for package in packages:
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])


In [ ]:
print('Importing Base Packages')
import requests
import json
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import matplotlib.pyplot as plt
import plotly.express as px
from io import BytesIO
import io
import os
from IPython.display import display, Image
import base64
import matplotlib.cm as cm
import seaborn as sns
from matplotlib.patches import Rectangle
import numpy as np
print('Completed Base Packages Import')

## Connecting with google API's

Connecting to google form and google sheet (hosted at Jason Ko's google account) with a HeadwaterOps specific google service account for non-user access.

I then print out the header of the google sheet to test whether the tunnel was successful

In [ ]:
print('defining OAuth scope')
# Step 1: Define the OAuth scope
scope = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive.file",
    "https://www.googleapis.com/auth/drive"
]

# Step 2: Authenticate using the downloaded JSON key
## Redacted service account .json file name

creds = ServiceAccountCredentials.from_json_keyfile_name("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%.json", scope)
client = gspread.authorize(creds)
print('Opening HOPS Response Sheet')
# Step 3: Open the sheet by name (must be shared with service account)
spreadsheet = client.open("HOPS Poker-Voting (Responses)")
sheet = spreadsheet.sheet1  # or use sheet = spreadsheet.worksheet("Form Responses 1")

# Step 4: Convert sheet to a DataFrame
print('Converting to DataFrame')
data = pd.DataFrame(sheet.get_all_records())
print(data.head())


data

## Mapping T-Shirt Size to Numerical Value

In [ ]:
size_map = {'S': 1, 'M': 2, 'L': 3, 'XL': 4, 'XXL': 5}

data = data.replace(size_map)

data

print('Calibrated T-Shirt to Numerical Map')
data

## Pushing Back to Monday.com

In [ ]:
## Redacted API Key and Board ID

MONDAY_API_KEY = "%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%"
BOARD_ID = "%%%%%%%%%%%%%%%%%%"  # Board where approved items live



headers = {
    "Authorization": MONDAY_API_KEY,
    "Content-Type": "application/json"
}
print('Connecting to MONDAY API and Board')

### Finding the column ID's exactly, for the voting board (unused, backend)

In [ ]:

query = """
query {
  boards(ids: %s) {
    columns {
      id
      title
    }
  }
}
""" % BOARD_ID

response = requests.post("https://api.monday.com/v2", json={"query": query}, headers=headers)
print(response.json())

### Log-style appending the voting board (duplicates are not removed)
#### Not executed, just defined

In [ ]:

def update_project_scores(project_name, impact, difficulty):
    project_name = project_name.strip()
    print(f"Looking for project: '{project_name}'")

    # Get all items from the board
    query = """
    query {
      boards(ids: %s) {
        items {
          id
          name
        }
      }
    }
    """ % BOARD_ID

    response = requests.post("https://api.monday.com/v2", json={"query": query}, headers=headers)
    res_json = response.json()
    print("Query response:", res_json)

    try:
        items = res_json['data']['boards'][0]['items']
        matching = [item for item in items if item['name'].strip() == project_name]
        if not matching:
            print(f"Project '{project_name}' not found.")
            return
        item_id = matching[0]['id']
    except (KeyError, IndexError):
        print("Error parsing response or project not found.")
        return

    # Build mutation to update columns
    mutation = {
        "query": """
        mutation {
          change_multiple_column_values(item_id: %s, board_id: %s, column_values: "{\\"impact\\": \\"%s\\", \\"difficulty\\": \\"%s\\"}") {
            id
          }
        }
        """ % (item_id, BOARD_ID, round(impact, 2), round(difficulty, 2))
    }

    update = requests.post("https://api.monday.com/v2", json=mutation, headers=headers)
    print(update.json())



def create_new_project_item(project_name, email_address, impact, difficulty):
    mutation = {
        "query": """
        mutation {
          create_item (
            board_id: %s,
            item_name: "%s",
            column_values: "{\\"text_mkt98crk\\": \\"%s\\", \\"text_mkt923w0\\": \\"%s\\", \\"text_mkt9f71h\\": \\"%s\\"}"
          ) {
            id
          }
        }
        """ % (BOARD_ID, project_name.strip(), email_address.strip(), round(impact, 2), round(difficulty, 2))
    }

    response = requests.post("https://api.monday.com/v2", json=mutation, headers=headers)
    print(f"Created item for {project_name}: {response.json()}")


    
# Run update for each row in aggregated
#for _, row in data.iterrows():
    #create_new_project_item(row["Project Name"], row["Email Address"], row["Impact Rating"], row["Difficulty Rating"])

In [ ]:
data

## Converting Itemized DataFrame of Votes into grouped by project

In [ ]:
print('Itemizing Votes')
average_ratingsDF = data.groupby('Project Name')[['Impact Rating', 'Difficulty Rating']].mean(numeric_only= True).reset_index()
average_ratingsDF.columns = ['Project Name', 'Impact Rating', 'Difficulty Rating']

average_ratingsDF

## Making Plot that uploads into each row

### Here I am splitting the big dataframe into many smaller dataframes sorted by project name. I am then iteratively plotting each of those smaller project dataframes individually

In [ ]:
data

In [ ]:
def create_scatter_plot_buffer5(vote_df, project_name):
    # Add jitter to points
    x = vote_df["Impact Rating"] + np.random.normal(0.05, 0.08, size=len(vote_df))
    y = vote_df["Difficulty Rating"] + np.random.normal(0.05, 0.08, size=len(vote_df))

    # Extract label names
    labels = vote_df["Your Name"].apply(lambda x: x.split('@')[0])

    # Map teams to colors
    teams = vote_df['Team'].unique()
    color_map = {team: color for team, color in zip(teams, plt.cm.tab10.colors)}
    vote_df['Color'] = vote_df['Team'].map(color_map)

    # Plot setup
    plt.figure(figsize=(6, 6))
    ax = plt.gca()

    # Add overlay rectangles
    ax.add_patch(Rectangle((3, 1), 2, 2, color='green', alpha=0.15))
    ax.add_patch(Rectangle((1, 3), 2, 2, color='red', alpha=0.15))

    # Scatter plot
    plt.scatter(x, y, c=vote_df['Color'], edgecolors='black', s=120)

    # Add labels
    for i, label in enumerate(labels):
        plt.annotate(label, (x.iloc[i] - 0.25, y.iloc[i] + 0.1), fontsize=10)

    # Axis settings
    size_labels = ['S', 'M', 'L', 'XL', 'XXL']
    ticks = [1, 2, 3, 4, 5]
    plt.xticks(ticks, size_labels)
    plt.yticks(ticks, size_labels)
    plt.xlim(0.5, 5.5)
    plt.ylim(0.5, 5.5)
    plt.xlabel("Impact")
    plt.ylabel("Lift")
    plt.grid(True)

    # Legend using dummy points
    for team in teams:
        plt.scatter([], [], color=color_map[team], label=team)
    plt.legend(title='Team')

    plt.title(f"Impact vs Difficulty Votes: {project_name}")
    plt.tight_layout()

    # Save to buffer
    buf = BytesIO()
    plt.savefig(buf, format='png')
    plt.close()
    buf.seek(0)
    return buf

In [ ]:
print('Split DataFrames to Group by project')

def create_scatter_plot_buffer(vote_df, project_name):
    x = vote_df["Impact Rating"] + np.random.normal(0.05, 0.08, size=len(vote_df["Impact Rating"]))
    y = vote_df["Difficulty Rating"] + np.random.normal(0.05, 0.08, size=len(vote_df["Difficulty Rating"]))
    #labels = vote_df["Email Address"]
    labels = vote_df["Your Name"].apply(lambda x: x.split('@')[0])
    plt.figure(figsize=(6, 6))
    # Add translucent green and red overlay rectangles:
    ax = plt.gca()
    # Example overlays:
    green_rect = Rectangle((3, 1), 2, 2, color='green', alpha=0.15)
    red_rect = Rectangle((1, 3), 2, 2, color='red', alpha=0.15)
    ax.add_patch(green_rect)
    ax.add_patch(red_rect)
    
    offsets = [
    (50, 30),   # top-right
    (-50, 30),  # top-left
    (50, -30),  # bottom-right
    (-50, -30), # bottom-left
    (0, 50),    # directly above
    (0, -50),   # directly below
    ] 

    teams = vote_df['Team'].unique()
    color_map = {team: color for team, color in zip(teams, plt.cm.tab10.colors)}

    # Add a color column to the dataframe
    vote_df['Color'] = vote_df['Team'].map(color_map)
    
    plt.scatter(x, y, c=vote_df['Color'], edgecolors='black', s=120)
    for i, label in enumerate(labels):
      ##  ox, oy = offsets[i % len(offsets)]  # Rotate through offset patterns
        plt.annotate(label, (x.iloc[i]-.25, y.iloc[i]+.1), fontsize=10)
     ##   plt.annotate(label,
     ####       textcoords="offset points",
      ##       xytext=(ox, oy),
       ##      ha='center', va='center',
       ##      arrowprops=dict(arrowstyle='->', lw=1))



    
    plt.title(f"Impact vs Lift Votes: {project_name}")

    size_labels = ['S', 'M', 'L', 'XL', 'XXL']
    plt.xticks(ticks=[1, 2, 3, 4, 5], labels=size_labels)
    plt.yticks(ticks=[1, 2, 3, 4, 5], labels=size_labels)    
    
    plt.xlabel("Impact")
    plt.ylabel("Lift")
    plt.xlim(0.5, 5.5)
    plt.ylim(0.5, 5.5)
    plt.grid(True, which = 'major')
 
    plt.legend(vote_df['Team'], title='Team')

    buf = BytesIO()
    plt.tight_layout()
    #plt.show()
    plt.savefig(buf, format='png')
    plt.close()
    buf.seek(0)  # Reset to beginning of the file
    return buf

def process_votes_and_upload(df):
    projects = df["Project Name"].unique()
    img_buffers = {}
    
    
    for project in projects:
        project_votes = df[df["Project Name"] == project]
        if project_votes.empty:
            continue
        #item_id = get_item_id_from_project(project)
        #if not item_id:
         #   print(f"Skipping {project}: item not found in Monday.")
        #    continue
        img_buffer = create_scatter_plot_buffer5(project_votes, project)
        img_buffers[project] = img_buffer
        #upload_file_to_item_from_buffer(item_id, img_buffer, f"{project.replace(' ', '_')}_votes.png")
    return img_buffers


In [ ]:
print('Processing Project-IND plots as buffer PNGs')    
buffers = process_votes_and_upload(data)

for project, buf in buffers.items():
    buf.seek(0)
    print(f"Project: {project}")
    display(Image(data=buf.read()))
    buf.seek(0)

## Storing these 'buffer' images into my old 'average_ratingsDF' pandas DF

In [ ]:
def get_image_buffer(project_name):
    buf = buffers.get(project_name)
    if buf:
        buf.seek(0)  # important to reset pointer for future use
    return buf

average_ratingsDF['Rate Plot'] = average_ratingsDF['Project Name'].map(get_image_buffer)
average_ratingsDF

# Upload the plots to github for link-storage

In [ ]:
## Redacted the GitHub token

GITHUB_TOKEN = "%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%"
GITHUB_REPO = "jamburs/LKCM_HOPS_Plots"
GITHUB_BRANCH = "main"

print('connecting to GitHub API for image storage, Project-Specific')

In [ ]:

def upload_image_to_github(image_buf, repo_path):
    image_buf.seek(0)
    content = base64.b64encode(image_buf.read()).decode("utf-8")

    url = f"https://api.github.com/repos/{GITHUB_REPO}/contents/{repo_path}"
    headers = {
        "Authorization": f"token {GITHUB_TOKEN}",
        "Accept": "application/vnd.github.v3+json"
    }

    # Check if file already exists to get its SHA
    sha = None
    check_response = requests.get(url, headers=headers)
    if check_response.status_code == 200:
        sha = check_response.json()["sha"]

    data = {
        "message": f"Add/update image {repo_path}",
        "content": content,
        "branch": GITHUB_BRANCH
    }
    if sha:
        data["sha"] = sha  # Needed for overwrite

    response = requests.put(url, headers=headers, json=data)
    response.raise_for_status()

    github_raw_url = f"https://raw.githubusercontent.com/{GITHUB_REPO}/{GITHUB_BRANCH}/{repo_path}"
    return github_raw_url

In [ ]:
for project, buf in buffers.items():
    filename = f"{project.replace(' ', '_')}_votes.png"
    github_path = f"plots/{filename}"  # Folder inside repo
    public_url = upload_image_to_github(buf, github_path)

    print(f"✅ Uploaded {filename} to GitHub at:\n{public_url}")
    # Now use this public_url in Monday.com update or column

### Adding the spat-out-link to the pandas DF to easily upload into monday

In [ ]:
# Store the links by project name
project_image_links = {}

for project, buf in buffers.items():
    filename = f"{project.replace(' ', '_')}_votes.png"
    github_path = f"plots/{filename}"
    
    try:
        public_url = upload_image_to_github(buf, github_path)
        project_image_links[project] = public_url
        print(f"✅ Uploaded for {project}: {public_url}")
    except Exception as e:
        print(f"❌ Failed to upload {project}: {e}")
        project_image_links[project] = None

def get_plot_link(project_name):
    return project_image_links.get(project_name)

# Create a new column in your existing dataframe
average_ratingsDF["GitHub Plot URL"] = average_ratingsDF["Project Name"].map(get_plot_link)

# Preview the result
average_ratingsDF[["Project Name", "GitHub Plot URL"]]
average_ratingsDF

# UPLOADING EVERYTHING INTO MONDAY

In [ ]:
print('Uploading averaged scores, plots into "In Progress" MONDAY Sheet')

## Redacted Board ID
BOARD2_ID = "%%%%%%%%%%%%%%%%"  # Board where approved items live

## column ID's:

In [ ]:

query = """
query {
  boards(ids: %s) {
    columns {
      id
      title
    }
  }
}
""" % BOARD2_ID

response = requests.post("https://api.monday.com/v2", json={"query": query}, headers=headers)
print(response.json())

In [ ]:
  query = """
    query {
      boards(ids: %s) {
        items_page {
          items {
            id
            name
          }
        }
      }
    }
    """ % BOARD2_ID

response = requests.post("https://api.monday.com/v2", json={"query": query}, headers=headers)
print(response.json())

In [ ]:
average_ratingsDF2 = average_ratingsDF

size_map = {1: 'S', 2: 'M', 3: 'L', 4: 'XL', 5: 'XXL'}
average_ratingsDF2['Impact Size'] = average_ratingsDF2['Impact Rating'].round().clip(1, 5).astype(int).map(size_map)
average_ratingsDF2['Difficulty Size'] = average_ratingsDF2['Difficulty Rating'].round().clip(1, 5).astype(int).map(size_map)

average_ratingsDF2

In [ ]:

def update_project_scores2(project_name, impact, difficulty, plotlink):
    project_name = project_name.strip()
    print(f"Looking for project: '{project_name}'")

    query = """
    query {
      boards(ids: %s) {
        items_page {
          items {
            id
            name
          }
        }
      }
    }
    """ % BOARD2_ID

    response = requests.post("https://api.monday.com/v2", json={"query": query}, headers=headers)
    res_json = response.json()
    print("Query response:", res_json)

    try:
        items = res_json['data']['boards'][0]['items_page']['items']
        matching = [item for item in items if item['name'].strip() == project_name]
        if not matching:
            print(f"Project '{project_name}' not found.")
            return
        item_id = matching[0]['id']
    except (KeyError, IndexError) as e:
        print("Error parsing response or project not found:", e)
        return

    # Proper JSON for column values
    column_values = {
        "text_mkt9ye4m": str(impact),        # Impact rating
        "text_mkt9rddj": str(difficulty),    # Difficulty rating
        "text_mktak41g": plotlink                      # GitHub plot URL
    }

    # GraphQL mutation using ID! (string) instead of Int!
    mutation = {
        "query": """
        mutation($item_id: ID!, $board_id: ID!, $column_values: JSON!) {
          change_multiple_column_values(
            item_id: $item_id,
            board_id: $board_id,
            column_values: $column_values
          ) {
            id
          }
        }
        """,
        "variables": {
            "item_id": item_id,                        # Leave as string
            "board_id": str(BOARD2_ID),                # Convert board_id to string
            "column_values": json.dumps(column_values)
        }
    }

    update = requests.post("https://api.monday.com/v2", json=mutation, headers=headers)
    print(update.json())

    
#for _, row in average_ratingsDF.iterrows():
#    update_project_scores2(row["Project Name"].split(' - ')[1], row["Impact Rating"], row["Difficulty Rating"], row["GitHub Plot URL"])

for _, row in average_ratingsDF2.iterrows():
    try:
        impact = row["Impact Size"]
        difficulty = row["Difficulty Size"]
        plot_url = row["GitHub Plot URL"]
        project = row["Project Name"].split(' - ')[1]

        update_project_scores2(project, impact, difficulty, plot_url)
    except Exception as e:
        print("Error processing row:", e)

## Pulling individual items from Monday API for consensus voting plot

## Only Ready for Review Projects

In [ ]:
## Redacted Board ID

BOARD4_ID = %%%%%%%%%%%%%%%%%%%%%%%%%%%%

query = """
query {
  boards(ids: [%d]) {
    items_page {
      items {
        id
        name
        column_values {
          id
          type
          text
          value
        }
      }
    }
  }
}
""" % float(BOARD4_ID)

url = "https://api.monday.com/v2"
headers = {
    "Authorization": MONDAY_API_KEY,
    "Content-Type": "application/json"
}

response = requests.post(url, headers=headers, json={"query": query})

data3 = response.json()
print(json.dumps(data3, indent=2))

In [ ]:
# --- Extract into a list of dicts for pandas ---
rows = []
for item in data3['data']['boards'][0]['items_page']['items']:
    row = {"Item ID": item["id"], "Item Name": item["name"]}
    for col in item["column_values"]:
        row[col["id"]] = col["text"]  # text value only
    rows.append(row)

# --- Create DataFrame ---
ConsensDF2 = pd.DataFrame(rows)

ConsensDF2

### Plotting Impact-Lift Map for Ready for Review Projects

In [ ]:
if 'ConsensDF2' in locals() and not ConsensDF2.empty:
    

    ConsensDF2 = ConsensDF2.rename(columns={'dropdown_mktqvgdn': 'ConsensusImpact', 'dropdown_mktqft80': 'ConsensusLift'})
    #ConsensDF
    size_map = {'S': 1, 'M': 2, 'L': 3, 'XL': 4, 'XXL': 5}

    ConsensDF2['ImpactNum'] = ConsensDF2.get('ConsensusImpact', pd.Series()).map(size_map)
    ConsensDF2['LiftNum'] = ConsensDF2.get('ConsensusLift', pd.Series()).map(size_map)

    #ConsensDF2['ImpactNum'] = ConsensDF2["ConsensusImpact"].map(size_map)
    #ConsensDF2['LiftNum'] = ConsensDF2["ConsensusLift"].map(size_map)


    ConsensDF2['Project Name'] = ConsensDF2['dropdown_mktgpyds'] + ' - ' + ConsensDF2['Item Name']



    
    print('Plotting Overview Project Scores - Again')

    project_names = ConsensDF2['Project Name']
    colors = cm.get_cmap('tab20', len(project_names))

    plt.figure(figsize=(8,6))
    ax = plt.gca()

    # Example overlay areas (adjust coordinates as you like):
    # Green area: Impact [4 to 6], Difficulty [0 to 2]
    green_rect = Rectangle((3, 0), 3, 3, color='green', alpha=0.15)
    ax.add_patch(green_rect)

    # Red area: Impact [0 to 2], Difficulty [4 to 6]
    red_rect = Rectangle((0, 3), 3, 3, color='red', alpha=0.15)
    ax.add_patch(red_rect)

    # Scatter plot points
    for i, project in enumerate(project_names):
        data = ConsensDF2[ConsensDF2['Project Name'] == project]
        #print(data)
        plt.scatter(
            #data['Average Impact Rating'], data['Average Difficulty Rating'],
            data['ImpactNum'] + np.random.normal(.04, 0.05, size=len(data['ImpactNum'])), data['LiftNum'] + np.random.normal(.04, 0.05, size=len(data['LiftNum'])),
            label=project,
            color=colors(i),
            s=120,
            edgecolor='black',
            linewidth=0.8,
            alpha=1,
            zorder=3,  # points above overlays
        )

    plt.xlim(0.5,5.5)
    plt.ylim(0.5,5.5)

    size_labels = ['S', 'M', 'L', 'XL', 'XXL']
    plt.xticks(ticks=[1, 2, 3, 4, 5], labels=size_labels)
    plt.yticks(ticks=[1, 2, 3, 4, 5], labels=size_labels)   

    plt.title("All Project Impact-Lift Map - Ready For Review")
    plt.xlabel("Impact Rating")
    plt.ylabel("Lift Rating")
    plt.grid(True, zorder=0)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    OverBuf2 = BytesIO()
    plt.savefig(OverBuf2, format='png')
    plt.show()
    

In [ ]:
## Redacted GitHub token

GITHUB_TOKEN = "%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%"
GITHUB_REPO = "jamburs/LKCM_HOPS_Plots"
GITHUB_BRANCH = "main"

print('connecting to GitHub API for image storage, OVERVIEW PLOT R4R')


In [ ]:
filename3 = "ReadyForReview_All_Project_Impact-Difficulty_Map.png"
github_path = f"plots/{filename3}"  # Folder inside repo


if 'OverBuf2' in locals() and OverBuf2.getbuffer().nbytes > 0:
    public_url = upload_image_to_github(OverBuf2, github_path)
    print(f"✅ Uploaded {filename3} to GitHub at:\n{public_url}")

# Now use this public_url in Monday.com update or column

## Only Approved / In Progress Projects

In [ ]:
## Redacted Board ID

BOARD3_ID = '%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%'

query = """
query {
  boards(ids: [%d]) {
    items_page {
      items {
        id
        name
        column_values {
          id
          type
          text
          value
        }
      }
    }
  }
}
""" % float(BOARD3_ID)

url = "https://api.monday.com/v2"
headers = {
    "Authorization": MONDAY_API_KEY,
    "Content-Type": "application/json"
}

response = requests.post(url, headers=headers, json={"query": query})

data2 = response.json()
print(json.dumps(data2, indent=2))

In [ ]:
# --- Extract into a list of dicts for pandas ---
rows = []
for item in data2['data']['boards'][0]['items_page']['items']:
    row = {"Item ID": item["id"], "Item Name": item["name"]}
    for col in item["column_values"]:
        row[col["id"]] = col["text"]  # text value only
    rows.append(row)

# --- Create DataFrame ---
ConsensDF = pd.DataFrame(rows)

print(ConsensDF.head())

In [ ]:
len(ConsensDF)

In [ ]:
if len(ConsensDF) != 0:

    ConsensDF = ConsensDF.rename(columns={'dropdown_mktqdk5z': 'ConsensusImpact', 'dropdown_mktqz9rr': 'ConsensusLift'})
    #ConsensDF
    size_map = {'S': 1, 'M': 2, 'L': 3, 'XL': 4, 'XXL': 5}
    ConsensDF['ImpactNum'] = ConsensDF["ConsensusImpact"].map(size_map)
    ConsensDF['LiftNum'] = ConsensDF["ConsensusLift"].map(size_map)


    ConsensDF['Project Name'] = ConsensDF['dropdown_mktg82f3'] + ' - '+  ConsensDF['Item Name']
    print(ConsensDF)

### Impact-Lift map for Approved/ In Progress

In [ ]:
if 'ConsensDF' in locals() and not ConsensDF.empty:

    print('Plotting Overview Project Scores')

    project_names = ConsensDF['Project Name']
    colors = cm.get_cmap('tab20', len(project_names))

    plt.figure(figsize=(8,6))
    ax = plt.gca()

    # Example overlay areas (adjust coordinates as you like):
    # Green area: Impact [4 to 6], Difficulty [0 to 2]
    green_rect = Rectangle((3, 0), 3, 3, color='green', alpha=0.15)
    ax.add_patch(green_rect)

    # Red area: Impact [0 to 2], Difficulty [4 to 6]
    red_rect = Rectangle((0, 3), 3, 3, color='red', alpha=0.15)
    ax.add_patch(red_rect)

    # Scatter plot points
    for i, project in enumerate(project_names):
        data = ConsensDF[ConsensDF['Project Name'] == project]
        plt.scatter(
            #data['Average Impact Rating'], data['Average Difficulty Rating'],
            data['ImpactNum'] + np.random.normal(.04, 0.05, size=len(data['ImpactNum'])), data['LiftNum'] + np.random.normal(.04, 0.05, size=len(data['LiftNum'])),
            label=project,
            color=colors(i),
            s=120,
            edgecolor='black',
            linewidth=0.8,
            alpha=1,
            zorder=3,  # points above overlays
        )

    plt.xlim(0.5,5.5)
    plt.ylim(0.5,5.5)

    size_labels = ['S', 'M', 'L', 'XL', 'XXL']
    plt.xticks(ticks=[1, 2, 3, 4, 5], labels=size_labels)
    plt.yticks(ticks=[1, 2, 3, 4, 5], labels=size_labels)   

    plt.title("All Project Impact-Lift Map - Approved/InProgress")
    plt.xlabel("Impact Rating")
    plt.ylabel("Lift Rating")
    plt.grid(True, zorder=0)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    OverBuf = BytesIO()
    plt.savefig(OverBuf, format='png')
    plt.show()

#### Uploading image to github

In [ ]:
## Redacted GitHub Token

GITHUB_TOKEN = "%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%"
GITHUB_REPO = "jamburs/LKCM_HOPS_Plots"
GITHUB_BRANCH = "main"

print('connecting to GitHub API for image storage, OVERVIEW PLOT')


In [ ]:
def upload_image_to_github2(image_buf, repo_path):
    image_buf.seek(0)
    content = base64.b64encode(image_buf.read()).decode("utf-8")

    url = f"https://api.github.com/repos/{GITHUB_REPO}/contents/{repo_path}"
    headers = {
        "Authorization": f"token {GITHUB_TOKEN}",
        "Accept": "application/vnd.github.v3+json"
    }

    # Check if file already exists to get its SHA
    sha = None
    check_response = requests.get(url, headers=headers)
    if check_response.status_code == 200:
        sha = check_response.json()["sha"]

    data = {
        "message": f"Add/update image {repo_path}",
        "content": content,
        "branch": GITHUB_BRANCH
    }
    if sha:
        data["sha"] = sha  # Needed for overwrite

    response = requests.put(url, headers=headers, json=data)
    response.raise_for_status()

    github_raw_url = f"https://raw.githubusercontent.com/{GITHUB_REPO}/{GITHUB_BRANCH}/{repo_path}"
    return github_raw_url


filename2 = "Approved/Progress_All_Project_Impact-Difficulty_Map.png"

if 'OverBuf' in locals() and OverBuf.getbuffer().nbytes > 0:
    github_path = f"plots/{filename2}"  # Folder inside repo
    public_url = upload_image_to_github(OverBuf, github_path)
    print(f"✅ Uploaded {filename2} to GitHub at:\n{public_url}")


# Now use this public_url in Monday.com update or column

In [ ]:
print("\n"+ "Execution Success" + "\n" *5 + "%" * 50 + "\n"*2 + "Job Complete")
# In[ ]: